# Analysis of the Evian water

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on March 31th, 2022.</p>

```{attention}
Always make sure you are using the [latest version of Reaktoro](https://anaconda.org/conda-forge/reaktoro). Otherwise, some new features documented on this website will not work on your machine and you may receive unintuitive errors. Follow these [update instructions](updating_reaktoro_via_conda) to get the latest version of Reaktoro!
```

This tutorial demonstrates the use of Reaktoro to calculate the saturation indices of minerals for a water sample with composition and pH shown below:

|![Evian water label](../../images/applications/evian-chemical-water-composition.png)|
|:--:|
|Evian water label, Source: wikipedia.org|

We start by defining the chemical system with just an aqueous phase:

In [20]:
from reaktoro import *
from math import *

db = PhreeqcDatabase("phreeqc.dat")

# Create an aqueous phase whose species are automatically determined from given list of elements
solution = AqueousPhase(speciate("C Ca Cl K Mg N Na S Si"))
solution.set(ActivityModelPhreeqc(db))  # Ensure consistency with PHREEQC by using the same activity model PHREEQC would use with the phreeqc.dat database!

# Create the chemical system with only the aqueous phase
system = ChemicalSystem(db, solution)

Next, we create an initial chemical state (in disequilibrium) representative of the water sample we are analysing:

In [ ]:
state = ChemicalState(system)
state.set("H2O"   , 1.000, "kg")
state.set("Ca+2"  , 80.00, "mg")
state.set("Cl-"   , 6.800, "mg")
state.set("HCO3-" , 350.0, "mg")
state.set("Mg+2"  , 26.00, "mg")
state.set("NO3-"  , 3.700, "mg")
state.set("K+"    , 1.000, "mg")
state.set("Na+"   , 6.500, "mg")
state.set("SO4-2" , 12.60, "mg")
state.set("H4SiO4", 24.00, "mg")

Note that the PHREEQC database `phreeqc.dat` does not have any aqueous species with formula `SiO2`. Therefore, we had to convert 15 mg of `SiO2`, with molar mass of 60.0843 g/mol, to the mass of `H4SiO4`, which has a molar mass of 96.1163 g/mol, in order to obtain the same molar amount of Si as in 15 mg of `SiO2`. 

For our speciation calculation, we require an equilibrium solver that allow us to specify not only temperature and pressure but also the pH of the solution, which is done next:

In [ ]:
# Specify the constraints we want to impose for the equilibrium calculation
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

# Create our equilibrium solver with the specifications above
solver = EquilibriumSolver(specs)

We can now equilibrate our initial state and determine its full composition:

In [21]:
# The conditions to be satisfied at the chemical equilibrium state
conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.pressure(1.0, "atm")
conditions.pH(7.2)

# Equilibrate the solution with the given initial chemical state and desired conditions at equilibrium
result = solver.solve(state, conditions)

assert result.succeeded()

We can now evaluate the aqueous properties of the solution and determine the saturation indices of the minerals and gases:

In [25]:
aprops = AqueousProps(state)
print(aprops)

+--------------------------------------+-------------+-------+
| Property                             |       Value |  Unit |
+--------------------------------------+-------------+-------+
| Temperature                          |    298.1500 |     K |
| Pressure                             |      1.0132 |   bar |
| Ionic Strength (Effective)           |      0.0089 | molal |
| Ionic Strength (Stoichiometric)      |      0.0090 | molal |
| pH                                   |      7.2000 |       |
| pE                                   |     12.1304 |       |
| Eh                                   |      0.7176 |     V |
| Charge Molality                      |  8.1165e-04 | molal |
| Element Molality:                    |             |       |
| :: C                                 |  5.7358e-03 | molal |
| :: N                                 |  5.9670e-05 | molal |
| :: Na                                |  2.8274e-04 | molal |
| :: Mg                                |  1.0695e-03 | 

The code below determines which minerals have a *super-saturated* state in the water sample (i.e., SI > 0):

In [35]:
print("Minerals for which the water is super-saturated:")
for species in aprops.saturationSpecies():
    name = species.name()
    SI = aprops.saturationIndex(name)
    if SI > 0.0:
        print(" - " + name)

Minerals for which the water is super-saturated:
 - Quartz
 - Dolomite
 - Calcite
